<a href="https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Cascading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://withpi.ai"><img src="https://play.withpi.ai/logo/logoFullBlack.svg" width="240"></a>

<a href="https://code.withpi.ai"><font size="4">Documentation</font></a>

<a href="https://play.withpi.ai"><font size="4">Technique Catalog</font></a>

# Cascading

Let's say you have two models, an expensive, slow, but high quality one, and a fast, cheap, but relatively lower quality one.

You can improve both cost and performance by using the faster model for simple tasks and the slower model for complex ones.  The trick is to know which input is which.  Pi Scorer can help with that.

## Install and initialize SDK

Connect to a regular CPU Python 3 runtime.  You won't need GPUs for this notebook.

You'll need a WITHPI_API_KEY from https://build.withpi.ai/account.  Add it to your notebook secrets (the key symbol) on the left.

Run the cell below to install packages and load the SDK

In [3]:
%%capture

%pip install withpi withpi-utils litellm datasets

import os
from google.colab import userdata
from withpi import PiClient

# Load the notebook secret into the environment so the Pi Client can access it.
os.environ["WITHPI_API_KEY"] = userdata.get('WITHPI_API_KEY')

pi = PiClient()

# Load a Scoring spec and a Dataset

We'll keep using a pre-built scoring spec with sample inputs, but feel free to bring your own.


In [2]:
# @title Load Scoring Spec
from withpi_utils.colab import load_scoring_spec_from_web, display_scoring_spec

aesop_scoring_spec = load_scoring_spec_from_web(
    "https://raw.githubusercontent.com/withpi/cookbook-withpi/refs/heads/main/scoring_specs/aesop_ai.json"
)

display_scoring_spec(aesop_scoring_spec)

## Load Dataset

We've published a small dataset on Hugging Face.  You should get a token from https://huggingface.co/settings/tokens and set it as a notebook secret called `HF_TOKEN`

In [5]:
from datasets import load_dataset

aesop_dataset = load_dataset("withpi/aesop", split="train")

print(aesop_dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 23
})


## Evaluate the scoring spec on different models.

Let's try generating responses from a "big" model and a "small" one to compare scores.

The below example uses Gemini because you can get a free key on the left pane (select Gemini API keys) or bring your own.
 https://docs.litellm.ai/docs/ describes the syntax for doing this.

In [8]:
from withpi_utils.colab import pretty_print_responses
import litellm

os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

aesop_application_description = """
Write a children's story in the style of Aesop's Fables teaching a life lesson
specified by the user. Provide just the story with no extra content.
"""

def generate(system: str, user: str, model: str) -> str:
    """generate passes the provided system and user prompts into the given model
    via LiteLLM"""
    messages = [
        {"content": system, "role": "system"},
        {"content": user, "role": "user"},
    ]
    return litellm.completion(model=model, messages=messages).choices[0].message.content


for i in range(5):
    row = aesop_dataset[i]
    small_model_output = generate(
        system=aesop_application_description,
        user=row["input"],
        model="gemini/gemini-2.0-flash-lite",
    )
    big_model_output = generate(
        system=aesop_application_description,
        user=row["input"],
        model="gemini/gemini-2.0-flash",
    )

    small_score = pi.scoring_system.score(
        llm_input=row["input"],
        llm_output=small_model_output,
        scoring_spec=aesop_scoring_spec,
    )
    big_score = pi.scoring_system.score(
        llm_input=row["input"],
        llm_output=big_model_output,
        scoring_spec=aesop_scoring_spec,
    )

    pretty_print_responses(
        header="#### Input:\n" + row["input"],
        response1="#### Output:\n" + small_model_output,
        response2="#### Output:\n" + big_model_output,
        left_label="gemini/gemini-2.0-flash-lite",
        right_label="gemini/gemini-2.0-flash",
        scores_left=small_score,
        scores_right=big_score,
    )
    print("\n\n")

"Does the story have a clear beginning, middle, and end?",1.0,
Is there a conflict introduced early in the story that drives the plot?,0.291,
Is the resolution of the conflict clear and satisfying?,0.801,
Does the story include characters that are relatable for children?,0.742,
Do the characters demonstrate growth or change by the end of the story?,0.27,
Is the dialogue between characters natural and age-appropriate?,0.73,
What makes a story engaging for children?,0.633,
Is the story engaging and likely to hold a child's interest?,0.719,
Does the story use vivid imagery to help children visualize the scenes?,0.428,
Does the story incorporate repetitive elements that aid in comprehension and retention?,0.672,
How can language be tailored for children's understanding?,0.738,


"Does the story have a clear beginning, middle, and end?",1.0,
Is there a conflict introduced early in the story that drives the plot?,0.322,
Is the resolution of the conflict clear and satisfying?,0.531,
Does the story include characters that are relatable for children?,0.617,
Do the characters demonstrate growth or change by the end of the story?,0.248,
Is the dialogue between characters natural and age-appropriate?,0.727,
What makes a story engaging for children?,0.621,
Is the story engaging and likely to hold a child's interest?,0.684,
Does the story use vivid imagery to help children visualize the scenes?,0.668,
Does the story incorporate repetitive elements that aid in comprehension and retention?,0.699,
How can language be tailored for children's understanding?,0.652,


"Does the story have a clear beginning, middle, and end?",1.0,
Is there a conflict introduced early in the story that drives the plot?,0.969,
Is the resolution of the conflict clear and satisfying?,1.0,
Does the story include characters that are relatable for children?,0.648,
Do the characters demonstrate growth or change by the end of the story?,0.676,
Is the dialogue between characters natural and age-appropriate?,0.973,
What makes a story engaging for children?,0.977,
Is the story engaging and likely to hold a child's interest?,0.812,
Does the story use vivid imagery to help children visualize the scenes?,0.918,
Does the story incorporate repetitive elements that aid in comprehension and retention?,0.797,
How can language be tailored for children's understanding?,0.746,


"Does the story have a clear beginning, middle, and end?",1.0,
Is there a conflict introduced early in the story that drives the plot?,0.498,
Is the resolution of the conflict clear and satisfying?,0.766,
Does the story include characters that are relatable for children?,0.707,
Do the characters demonstrate growth or change by the end of the story?,0.381,
Is the dialogue between characters natural and age-appropriate?,0.73,
What makes a story engaging for children?,0.738,
Is the story engaging and likely to hold a child's interest?,0.734,
Does the story use vivid imagery to help children visualize the scenes?,0.504,
Does the story incorporate repetitive elements that aid in comprehension and retention?,0.699,
How can language be tailored for children's understanding?,0.73,


"Does the story have a clear beginning, middle, and end?",1.0,
Is there a conflict introduced early in the story that drives the plot?,0.68,
Is the resolution of the conflict clear and satisfying?,0.664,
Does the story include characters that are relatable for children?,0.809,
Do the characters demonstrate growth or change by the end of the story?,0.707,
Is the dialogue between characters natural and age-appropriate?,0.711,
What makes a story engaging for children?,0.738,
Is the story engaging and likely to hold a child's interest?,0.715,
Does the story use vivid imagery to help children visualize the scenes?,0.688,
Does the story incorporate repetitive elements that aid in comprehension and retention?,0.602,
How can language be tailored for children's understanding?,0.695,


## Next Steps

Now you can imagine unconditionally applying a cheap model, scoring it, then if it scores below a threshold falling back to the big model.

An improvement to this technique is to train a classifier that can predict whether a particular prompt is likely to require a more powerful model.  Then start with the big one if warranted. A fully trained classifier is ideal, but a simple scorer with a few smart questions may suffice.